### Count all publications which their pubdates are null

In [ ]:
import duckdb

paragraph_md5sum_file = "/work/data/projects/data2report/paragraph-md5sum-2025012501.duckdb"
conn = duckdb.connect(paragraph_md5sum_file, read_only=True)

conn.execute(f"""
SELECT COUNT(*)
FROM publications
WHERE pubdate is null;
""").fetchall()

### Show all indexes

In [ ]:
import duckdb

paragraph_md5sum_file = "/work/data/projects/data2report/paragraph-md5sum-2025012501.duckdb"
conn = duckdb.connect(paragraph_md5sum_file)

conn.execute(f"""
SELECT *
FROM duckdb_indexes 
WHERE table_name = 'publications';
""").df()

### Explain SQL Plan 1

In [ ]:
import duckdb
import json
import pandas as pd

paragraph_md5sum_file = "/work/data/projects/data2report/paragraphs-md5sum-2025012501/md5sum_prefix=*/**.parquet"
conn = duckdb.connect()

keys = json.load(open("/work/data/projects/data2report/keys.json"))
md5sum_prefixes = [k[:2] for k in keys]

print("MD5SUM_PREFIXES: ", len(md5sum_prefixes), len(keys))

keys_df = pd.DataFrame({"md5sum": keys})  # keys 是要查询的 md5sum 列表
conn.register("temp_keys", keys_df)

md5sum_prefixes_df = pd.DataFrame({"md5sum_prefix": md5sum_prefixes})
conn.register("temp_md5sum_prefixes", md5sum_prefixes_df)

conn.execute(f"""
    EXPLAIN
    SELECT p.md5sum, p.text, p.pmid, p.title
    FROM read_parquet('{paragraph_md5sum_file}') p
    INNER JOIN temp_keys k ON p.md5sum = k.md5sum 
    INNER JOIN temp_md5sum_prefixes mp ON p.md5sum_prefix = mp.md5sum_prefix
""").fetchall()

### Explain SQL Plan 2

In [ ]:
import json

paragraph_md5sum_file = "/work/data/projects/data2report/paragraphs-md5sum-2025012501/md5sum_prefix=*/**.parquet"
conn = duckdb.connect()

keys = json.load(open("/work/data/projects/data2report/keys.json"))

# 将 keys 转换为临时表
conn.execute("CREATE TEMP TABLE tmp_keys (md5sum VARCHAR);")
conn.executemany("INSERT INTO tmp_keys VALUES (?)", [(k,) for k in keys])

# 使用 JOIN 查询
conn.execute("""
    EXPLAIN
    SELECT p.md5sum, p.text, p.pmid, p.title
    FROM publications p
    JOIN tmp_keys k ON p.md5sum = k.md5sum
""").fetchall()

In [ ]:
# conn.execute(f"SHOW TABLES;").fetchall()

keys_df = pd.DataFrame({"md5sum": keys})  # keys 是要查询的 md5sum 列表
conn.register("temp_keys", keys_df)

# 直接 JOIN 查询
metadata = conn.execute("""
    SELECT p.md5sum, p.text, p.pmid, p.title
    FROM publications p
    INNER JOIN temp_keys k ON p.md5sum = k.md5sum
""").fetch_df()
metadata

### Explain SQL Plan 3

In [ ]:
import duckdb
import json

paragraph_md5sum_file = "/work/data/projects/data2report/paragraphs-md5sum-2025012501/md5sum_prefix=*/**.parquet"
conn = duckdb.connect()

keys = json.load(open("/work/data/projects/data2report/keys.json"))
md5sum_prefixes = [k[:2] for k in keys]

print("MD5SUM_PREFIXES: ", len(md5sum_prefixes), len(keys))

conn.execute(f"""
    EXPLAIN
    SELECT md5sum, text, pmid, title
    FROM read_parquet('{paragraph_md5sum_file}')
    WHERE md5sum IN ({", ".join(map(repr, keys))}) AND md5sum_prefix IN ({", ".join(map(repr, md5sum_prefixes))})
""").fetchall()